# PS Parallel Programming / Sheet 08
# Fabio Valentini / MN 01018782

## Exercise 1

### Snippet 1

```c
a[0] = 0;
#pragma omp parallel for
for (i=1; i<N; i++) {
    a[i] = 2.0*i*(i-1);     // S1
    b[i] = a[i] - a[i-1];   // S2
}
```

#### Correct Parallelization

No.

#### Dependency Analysis

The first access to `a[i-1]` in each parallel chunk might depend on uninitialized memory, because it is the last number that is written to `a` in the previous chunk. This is a true dependence $ S_1 \delta S_2$.

#### Potential Parallel Fixes

##### 1. Resolve true dependence

Splitting the initialization of data in `a` and the calculation of `b` into separate parallel loops would resolve the true dependence, because all elements of `a` would definitely be written before they are read in the second parallel loop.

```c
a[0] = 0;
#pragma omp parallel for
for (i=1; i<N; i++) {
    a[i] = 2.0*i*(i-1);     // S1
}
#pragma omp parallel for
for (i=1; i<N; i++) {
    b[i] = a[i] - a[i-1];   // S2
}
```

##### 2. Change algorithm for calculation of `b`

The data in `a` is only dependent on the array index `i`, so it would be possible to calculate the data needed for elements of `b` "on the fly" without the helper array `a`, thus completely eliminating the need for a second array and no chance of data races (by solving the recurrence relation):

```c
#pragma omp parallel for
for (i=1; i<N; i++) {
    b[i] = 4.0 * i - 4;
}
```

### Snippet 2

```c
a[0] = 0;
#pragma omp parallel
{
    #pragma omp for nowait
    for (i=1; i<N; i++) {
        a[i] = 3.0*i*(i+1);    // S1
    }
    #pragma omp for
    for (i=1; i<N; i++) {
        b[i] = a[i] - a[i-1];  // S2
    }
}
```

#### Correct Parallelization

No. Same problem as in Snippet 1, because of the `nowait` clause.

The true dependence $S_1 \delta S_2$ is not resolved, because threads of the second loop might be started before threads in the first loop are finished - due to the `nowait` clause, there is no implicit barrier at the end of the for loop.

#### Dependency Analysis

True dependence $S_1 \delta S_2$ (see Snippet 1).

#### Potential Parallel Fixes

##### 1. Dropping `nowait` clause

Dropping the `nowait` clause of the first parallel for loop resolves the dependency, because then the implicit barrier at the end of the for loop forces all elements of `a` to be initialized before the second for loop accesses them.

##### 2. Change algorithm for calculation of b

The recurrence relation can be solved in this case too, leading to a simpler algorithm that does not need array `a` or any synchroniation at all:

```c
#pragma omp parallel for
for (i=1; i<N; i++) {
    b[i] = 6.0 * i;
}
```

### Snippet 3

```c
#pragma omp parallel for
for (i=1; i<N; i++) {
    x = sqrt(b[i]) - 1;    // S1
    a[i] = x*x + 2*x + 1;  // S2
}
```

#### Correct Parallelization

Yes (even if pointers `a` and `b` are aliases).

#### Dependency Analysis

Array `b` is only read from, array `a` is only written to. There are no potential problems.

Even if pointers `a` and `b` are aliases, there is only an antidependence $S_1 \delta^{-1} S_2$ of array elements with the **same** index `i` in each loop, which is unproblematic.

#### Potential Parallel Fixes

None.

### Snippet 4

```c
f = 2;
#pragma omp parallel for private(f,x)
for (i=1; i<N; i++) {
    x = f * b[i];    // S1
    a[i] = x - 7;    // S2
}
a[0] = x; 
```

#### Correct Parallelization

No. The value of `private(x)` is undefined after the parallel section, so the value of `a[0]` is undefined. The value of `private(f)` is also undefined after entering the parallel region.

#### Dependency Analysis

There is only an unproblematic antidependence $S_1 \delta^{-1} S_2$ between array elements with the **same** index `i` of `a` and `b`, so executing in parallel causes no problems.

#### Potential Parallel Fixes

- use `firstprivate(f)` so `f` will retain its value of `2` from outside the parallel section
- use `lastprivate(x)` so `x` will retain its last valid value from inside the parallel region (i.e. from the loop execution with index `i = N-1`)

On the other hand, `f` could just be `shared`, as it is not modified inside the loop body.

### Snippet 5

```c
sum = 0;
#pragma omp parallel for
for (i=1; i<N; i++) {
    sum = sum + b[i];    // S1
}
```

#### Correct Parallelization

No. There are possible data races during the update of `sum`.

#### Dependency Analysis

There is a true dependence $S_1 \delta S_1$ and an antidependence $S_1 \delta^{-1} S_1$ between different loop executions, as the shared `sum` is both written to and read from every time, regardless of loop counter, making all executions of the loop body depend on all others.

#### Potential Parallel Fixes

##### 1. use local partial sums and synchronize access to global `sum` (see exercises from week 01)

```c
sum = 0;

#pragma omp parallel
{
    _ local_sum = 0;

    #pragma omp for
    for (i=1; i<N; i++) {
        local_sum = local_sum + b[i];  // S1
    }

    #pragma omp critical
    sum += local_sum;
}
```

##### 2. Use reduction functionality provided by OpenMP

```c
sum = 0;
#pragma omp parallel for reduction (+: sum)
for (i=1; i<N; i++) {
    sum = sum + b[i];    // S1
}
```

This version has the benefit of being very simple, and possibly benefiting from an optimized summation algorithm provided by OpenMP, but should otherwise translate to similar code as fixed snippet 1, using local partial sums.

## Exercise 2

The source code of `analysis.c` was annotated with GCC 8.2.0 compiler output when compiling with `-O2 -ftree-vectorize -fopt-info-vec-all`. Note that compiler output was in reverse order (last loop in source file considered first), so the order in the annotated source code does not directly correspond to the compiler output.

```c
#include <stdio.h>

#define SIZE 1024

int main(int argc, char** argv) {

        int a[SIZE];
        int b[SIZE];

        for(int i = 0; i < SIZE; ++i) {
                a[i] = argc;
        }
```

    Analyzing loop at analysis.c:10
    [...]
    analysis.c:10:9: note: step: 1,  init: 0
    [...]
    analysis.c:10:9: note: vectype: vector(4) int
    analysis.c:10:9: note: nunits = 4
    [...]
    analysis.c:10:9: note: vectorization_factor = 4, niters = 1024
    [...]
    analysis.c:10:9: note: recording new base alignment for &a
    analysis.c:10:9: note:   alignment:    16
    analysis.c:10:9: note:   misalignment: 0
    [...]
    analysis.c:10:9: note: Cost model analysis: 
      Vector inside of loop cost: 12
      Vector prologue cost: 4
      Vector epilogue cost: 0
      Scalar iteration cost: 12
      Scalar outside cost: 0
      Vector outside cost: 4
      prologue iterations: 0
      epilogue iterations: 0
      Calculated minimum iters for profitability: 1
    analysis.c:10:9: note:   Runtime profitability threshold = 4
    analysis.c:10:9: note:   Static estimate profitability threshold = 4
    [...]
    analysis.c:10:9: note: New loop exit condition: if (ivtmp_66 < 256)
    analysis.c:10:9: note: LOOP VECTORIZED

- `step: 1,  init: 0`: corresponds to loop variable initialization and update pattern
- no dependence distance / information: `a` is only written to, and the written values are constants
- `vectorization_factor = 4, niters = 1024`: SIMD instructions for 4 values are chosen, 1024 loop iterations calculated
- `recording new base alignment for &a` and `alignment: 16` seems to make sure memory access is correctly aligned for SIMD operations
- `Cost model analysis`: compiler includes a const-benefit analysis of vectorizing the loop (probably based on heuristics and CPU specs / instruction "costs", if available); in this case, the benefit apparently outweighs the overhead even for very small arrays (>= 4 elements)
- `New loop exit condition: if (ivtmp_66 < 256)`: corresponds to 1024 / 4 loop iterations for transformed loop

```c
        for(int i = 0; i < SIZE; ++i) {
                b[i] = a[i];
        }
```

    Analyzing loop at analysis.c:14
    [...]
    analysis.c:14:9: note: Access function of PHI: {0, +, 1}_2
    analysis.c:14:9: note: step: 1,  init: 0
    [...]
    analysis.c:14:9: note: vectype: vector(4) int
    analysis.c:14:9: note: nunits = 4
    [...]
    analysis.c:14:9: note: vectorization_factor = 4, niters = 1024
    [...]
    analysis.c:14:9: note: recording new base alignment for &a
    analysis.c:14:9: note:   alignment:    16
    analysis.c:14:9: note:   misalignment: 0
    [...]
    analysis.c:14:9: note: recording new base alignment for &b
    analysis.c:14:9: note:   alignment:    16
    analysis.c:14:9: note:   misalignment: 0
    [...]
    analysis.c:14:9: note: Cost model analysis: 
      Vector inside of loop cost: 24
      Vector prologue cost: 0
      Vector epilogue cost: 0
      Scalar iteration cost: 24
      Scalar outside cost: 0
      Vector outside cost: 0
      prologue iterations: 0
      epilogue iterations: 0
      Calculated minimum iters for profitability: 0
    analysis.c:14:9: note:   Runtime profitability threshold = 4
    analysis.c:14:9: note:   Static estimate profitability threshold = 4
    [...]
    analysis.c:14:9: note: New loop exit condition: if (ivtmp_60 < 256)
    analysis.c:14:9: note: LOOP VECTORIZED

- `step: 1,  init: 0`: corresponds to loop variable initialization and update pattern
- no dependence distance / information: `a` and `b` are different arrays and no aliases (can be checked statically in this case)
- `vectorization_factor = 4, niters = 1024`: SIMD instructions for 4 values are chosen, 1024 loop iterations calculated
- `recording new base alignment for &a / &b` and `alignment: 16` seems to make sure memory access is correctly aligned for SIMD operations
- `Cost model analysis`: compiler includes a const-benefit analysis of vectorizing the loop (probably based on heuristics and CPU specs / instruction "costs", if available); in this case, the benefit apparently outweighs the overhead even for very small arrays (>= 4 elements)
- `New loop exit condition: if (ivtmp_60 < 256)`: corresponds to 1024 / 4 loop iterations for transformed loop

```c
        for(int i = 4; i < SIZE; ++i) {
                a[i-4] = a[i];
        }
```

    Analyzing loop at analysis.c:18
    [...]
    analysis.c:18:9: note: step: 1,  init: 4
    [...]
    analysis.c:18:9: note: dependence distance  = 4.
    analysis.c:18:9: note: dependence distance negative.
    [...]
    analysis.c:18:9: note: get vectype for scalar type:  int
    analysis.c:18:9: note: vectype: vector(4) int
    [...]
    analysis.c:18:9: note: vectorization_factor = 4, niters = 1020
    [...]
    analysis.c:18:9: note: accesses have the same alignment: a[i_51] and a[_2]
    analysis.c:18:9: note: recording new base alignment for &a
    analysis.c:18:9: note:   alignment:    16
    analysis.c:18:9: note:   misalignment: 0
    [...]
    analysis.c:18:9: note: Cost model analysis: 
      Vector inside of loop cost: 24
      Vector prologue cost: 0
      Vector epilogue cost: 0
      Scalar iteration cost: 24
      Scalar outside cost: 0
      Vector outside cost: 0
      prologue iterations: 0
      epilogue iterations: 0
      Calculated minimum iters for profitability: 0
    analysis.c:18:9: note:   Runtime profitability threshold = 4
    analysis.c:18:9: note:   Static estimate profitability threshold = 4
    [...]
    analysis.c:18:9: note: New loop exit condition: if (ivtmp_14 < 255)
    analysis.c:18:9: note: LOOP VECTORIZED

Some statements in this compiler output snippet are particularly interesting:

- `step: 1,  init: 4`: corresponds to loop variable initialization and update pattern
- `dependence distance  = 4`: corresponds to `a[i-4]` and `a[i]` access pattern
- `dependence distance negative`: array elements are only ever read before they are written to
- `vectorization_factor = 4, niters = 1020`: SIMD instructions for 4 values are chosen, 1020 loop iterations calculated
- `recording new base alignment for &a` and `alignment: 16` seems to make sure memory access is correctly aligned for SIMD operations
- `Cost model analysis`: compiler includes a const-benefit analysis of vectorizing the loop (probably based on heuristics and CPU specs / instruction "costs", if available); in this case, the benefit apparently outweighs the overhead even for very small arrays (>= 4 elements)
- `New loop exit condition: if (ivtmp_14 < 255)`: corresponds to 1020 / 4 loop iterations for transformed loop

```c
        for(int i = 1; i < SIZE-1; ++i) {
                a[i] = a[i%argc];
        }
```

    Analyzing loop at analysis.c:22
    [...]
    vector(4) int
    analysis.c:22:9: note: not vectorized: not suitable for gather load _5 = a[_4];
    analysis.c:22:9: note: bad data references.

Looks like GCC does not vectorize this loop because of unpredictable array access patterns (because they depend on the number of command line arguments, they cannot be analyzed statically).

```c
        // output data to prevent compiler from removing any code
        for(int i = 0; i < SIZE; ++i) {
                printf("%d ", a[i]);
                printf("%d ", b[i]);
        }
```

    Analyzing loop at analysis.c:27
    [...]
    analysis.c:27:9: note: not vectorized: loop contains function calls or data references that cannot be analyzed

It makes sense that the compiler would not vectorize this loop. I assume that it is never safe to vectorize calls to functions that write to file descriptors (in this case, `stdout`), as that would mess up the order in which data is written, even if the `printf` implementation were thread-safe and would flush buffers after every write.

```c
        printf("\n");

        return 0;
}
```

So, in addition to dependence analysis (signed distance of memory accesses, etc.), the compiler also included procedures to make sure data is properly aligned in memory for SIMD instructions, and a cost-benefit analysis for vectorizing loops vs. not vectorizing them.

## Exercise 3

### Snippet 1

```c
void copy(double* x, double* y) {
    for(int i = 0; i < 1024; i++) {
        x[i] = y[i];
    }
}
```

This code can not be safely parallelized by the compiler. `x` and `y` could be aliases, or could be pointing to overlapping memory regions, in which case there would be dependencies between read and write accesses in the overlapping region.

If the programmer can verify that `x` and `y` will not be pointing to overlapping memory regions, this can be safely parallelized; either by reasoning that the function is always called with valid arguments, or by running checks before attempting to copy:

```c
#define SIZE 1024

int copy(double* x, double* y) {
    if (x == y)
        // pointless
        return -1;

    if ((x > y && (y+SIZE >= x)) || (y > x && (x+SIZE >= y)))
        // overlapping memory regions
        return -1;

    #pragma omp parallel for
    for(int i = 0; i < SIZE; i++) {
        x[i] = y[i];
    }

    // successful copy
    return 0;
}
```

### Snippet 2

Normalize the following loop nest:

```c
for (int i=4; i<=N; i+=7) {
    for (int j=0; j<=N; j+=3) {
        A[i] = 0;
    }
}
```

- `i1`: $L_i = 4$, $U_i = N$, $S_i = 7$
- `i2`: $L_j = 0$, $U_j = N$, $S_j = 3$
- $i \rightarrow i_1 \cdot S_i - S_i + L_i = i_1 \cdot 7 - 7 + 4 = i_1 \cdot 7 - 3$
- $j \rightarrow i_2 \cdot S_j - S_j + L_j = i_2 \cdot 3 - 3 + 0 = i_2 \cdot 3 - 3$

- $L_{i_1} = 1$, $S_{i_1} = 1$
- $U_{i_1} = (U_i - L_i + S_i) / S_i = (N-4)/7 + 1$
- $L_{i_2} = 1$, $S_{i_2} = 1$
- $U_{i_2} = (U_j - L_j + S_j) / S_j = N/3 + 1$

```c
for (int i1 = 1; i1 <= ((N-4)/7)+1; i1++) {
    for (int i2 = 1; i2 <= (N/3)+1; i2++) {
        A[i1*7-3] = 0;
    }
}
```

### Snippet 3

Dependency analysis, distance vectors, possible parallelization:

```c
for(int i = 1; i < N; i++) {
    for(int j = 1; j < M; j++) {
        for(int k = 1; k < L; k++) {
            a[i+1][j][k-1] = a[i][j][k] + 5;    // S1
        }
    }
}
```

This loop next is already normalized, because `i<N` and `i<=N-1` etc. are equivalent.

There is a true dependence of statement `S1` on itself ($S_1 \delta S_1$, due to loop variable `i`), and an antidependence of `S1` on itself ($S_1 \delta^{-1} S_1$, due to loop variable `k`). For the true dependence, the distance vector evaluates to a constant $(+1, 0, -1)$, and the direction vector evaluates to $(<,=,>)$.

The outermost loop cannot be parallelized at all, due to the true dependence with distance 1. The innermost loop cannot be parallelized at all due to an antidependence with distance 1 (this could be resolved by using a temporary copy of the whole array `a`, which would allow to write elements "before they are read", because the "original" element can still be read from the temporary copy instead).

However, the middle loop can be safely parallelized, because memory locations of elements that are written to are always distinct from memory locations of elements that are read from (since the most-significant array index `i` is different), for example:

```c
for(int i = 1; i < N; i++) {
    # pragma omp parallel for
    for(int j = 1; j < M; j++) {
        for(int k = 1; k < L; k++) {
            a[i+1][j][k-1] = a[i][j][k] + 5;    // S1
        }
    }
}
```

Whether this parallelization provides a good performance improvement will probably depend on the size of the constants `N`, `M`, and `L`. If `M` is small compared to the other dimensions, then the benefit will probably be small. If `M` is the biggest dimension, then this parallelization should provide a decent speedup.